<a href="https://colab.research.google.com/github/Aswin2808/AI/blob/main/Simple_GAN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as L
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
import os

# Selected size for real data and generated images
IMG_SHAPE = (32, 32, 3)
# Selected size for image encoding
CODE_SIZE = 100  # Reduced for simplicity

class GAN:
    def __init__(self):
        """ Initialize GAN object. """
        self.global_epoch = 0
        self.IMG_SHAPE = IMG_SHAPE
        self.CODE_SIZE = CODE_SIZE

    def get_data(self):
        """ Load and preprocess CIFAR-10 dataset. """
        print('Loading CIFAR-10 dataset.')
        (X_train, _), (_, _) = cifar10.load_data()
        X_train = X_train.astype(np.float32) / 255.0
        self.data = X_train
        print('Data prepared. %d images loaded.' % X_train.shape[0])

    def create_generator(self):
        """ Model to generate images from random noise. """
        generator = Sequential()
        generator.add(L.Dense(8 * 8 * 128, activation='relu', input_dim=self.CODE_SIZE))
        generator.add(L.Reshape((8, 8, 128)))
        generator.add(L.Conv2DTranspose(128, kernel_size=5, strides=2, padding='same', activation='relu'))
        generator.add(L.Conv2DTranspose(64, kernel_size=5, strides=2, padding='same', activation='relu'))
        generator.add(L.Conv2D(3, kernel_size=7, padding='same', activation='sigmoid'))
        self.generator = generator
        print('Generator created successfully.')

    def create_discriminator(self):
        """ Model to distinguish real images from generated ones. """
        discriminator = Sequential()
        discriminator.add(L.Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=self.IMG_SHAPE))
        discriminator.add(L.LeakyReLU(alpha=0.2))
        discriminator.add(L.Conv2D(128, kernel_size=5, strides=2, padding='same'))
        discriminator.add(L.LeakyReLU(alpha=0.2))
        discriminator.add(L.Flatten())
        discriminator.add(L.Dense(1, activation='sigmoid'))
        self.discriminator = discriminator
        self.discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
        print('Discriminator created successfully.')

    def create_gan(self):
        """ Compile GAN model combining generator and discriminator. """
        self.create_generator()
        self.create_discriminator()

        self.discriminator.trainable = False

        gan_input = L.Input(shape=(self.CODE_SIZE,))
        x = self.generator(gan_input)
        gan_output = self.discriminator(x)

        self.gan = Sequential([self.generator, self.discriminator])
        self.gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
        print('GAN model created successfully.')

    def sample_noise_batch(self, bsize):
        """ Generate random noise for image generation. """
        return np.random.normal(size=(bsize, self.CODE_SIZE)).astype('float32')

    def sample_data_batch(self, bsize):
        """ Sample images from real data. """
        idxs = np.random.choice(np.arange(self.data.shape[0]), size=bsize)
        return self.data[idxs]

    def sample_images(self, nrow, ncol, epoch):
        """ Generate images from random noise and save to file. """
        noise = self.sample_noise_batch(nrow * ncol)
        images = self.generator.predict(noise)
        images = (images + 1) / 2.0  # Rescale images 0 - 1

        fig, axs = plt.subplots(nrow, ncol)
        cnt = 0
        for i in range(nrow):
            for j in range(ncol):
                axs[i, j].imshow(images[cnt])
                axs[i, j].axis('off')
                cnt += 1
        plt.suptitle('Epochs: ' + str(epoch))
        sample_dir = 'output'
        if not os.path.exists(sample_dir):
            os.makedirs(sample_dir)
        plt.savefig(sample_dir + '/render_epochs_' + str(epoch) + '.png', dpi=200)
        plt.close()

    def print_generated_image(self, noise):
        """ Print a single generated image. """
        image = self.generator.predict(noise[np.newaxis, :])[0]
        plt.imshow((image + 1) / 2.0)
        plt.axis('off')
        plt.show()

    def evaluate_image(self, image):
        """ Evaluate if the image is classified as real or fake by the discriminator. """
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        prediction = self.discriminator.predict(image)[0][0]
        return "Real" if prediction > 0.5 else "Fake"

    def train(self, epochs, batch_size, discriminator_steps=1, generator_steps=1):
        """ Perform training steps and get results. """
        for epoch in range(epochs):
            # Train Discriminator
            for _ in range(discriminator_steps):
                real_images = self.sample_data_batch(batch_size)
                real_labels = np.ones((batch_size, 1))
                fake_images = self.generator.predict(self.sample_noise_batch(batch_size))
                fake_labels = np.zeros((batch_size, 1))

                d_loss_real = self.discriminator.train_on_batch(real_images, real_labels)
                d_loss_fake = self.discriminator.train_on_batch(fake_images, fake_labels)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator
            for _ in range(generator_steps):
                noise = self.sample_noise_batch(batch_size)
                valid_labels = np.ones((batch_size, 1))
                g_loss = self.gan.train_on_batch(noise, valid_labels)

            if epoch % 100 == 0:
                self.sample_images(4, 4, self.global_epoch)
                print(f'Epoch: {self.global_epoch}, '
                      f'Generator loss: {g_loss}, '
                      f'Discriminator loss: {d_loss[0]}, accuracy: {100 * d_loss[1]}%')
            self.global_epoch += 1

# Start the process
gan = GAN()
gan.get_data()
gan.create_gan()
gan.train(epochs=1000, batch_size=64, discriminator_steps=1, generator_steps=1)

# Generate and evaluate a single image
noise = gan.sample_noise_batch(1)
gan.print_generated_image(noise[0])
generated_image = gan.generator.predict(noise)[0]
print("Discriminator output for generated image:", gan.evaluate_image(generated_image))


Loading CIFAR-10 dataset.
Data prepared. 50000 images loaded.
Generator created successfully.
Discriminator created successfully.
GAN model created successfully.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step  
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
Epoch: 0, Generator loss: [array(0.69633156, dtype=float32), array(0.69633156, dtype=float32), array(0.4921875, dtype=float32)], Discriminator loss: 0.6653913259506226, accuracy: 73.828125%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
